In [40]:
degBit = 11
degStable = 11
delta = 1/6

def binomial_pdf(n, p, x):
    return binomial(n, x) * p ^ x * (1 - p) ^ (n-x)

# Pr[X >= x]
def binomial_ncdf(n, p, x):
    probSum = 0.0
    for i in range(x, n + 1):
        probSum = probSum + binomial_pdf(n, p, i)
    return probSum

# phi(targetP) = Pr[X >= x], return x
def binomial_ncdf_inverse(n, p, targetP):
    for i in range(1, n):
        ncdf_p = binomial_ncdf(n, p, i)
        if ncdf_p < targetP:
            return i - 1
    return n


# https://stats.stackexchange.com/questions/394960/variance-of-normal-order-statistics
# Variance for quantile
# TODO: verify
def binomial_inverse_variance(N, p, inverse, M):
	return p * (1-p) / ((M+2) * (binomial_pdf(N, p, inverse)^2))

## Lower bound via taking advantage of discrete RV
# where V < mu
def chebyshevLowerBound(V, mu, variance):
	return min(variance / (mu - V)^2, 1)

## Upper bound via taking advantage of discrete RV
# where V > mu and we are looking for Pr[mu < V]
def chebyshevUpperBound(V, mu, variance):
	return 1 - min(variance / (mu - V)^2, 0)

def pr_G_error_given_E(E, degBit, degStable, delta, k, M, pG):
	# a lower bound for the expectation of G
	nStabilizersForG = degBit * degStable - degStable * E
	nStabilizersForGPrime = floor((1 - delta) * degBit * degStable) - degStable * E

	if nStabilizersForG <= 0:
		return 1
	expectedG = binomial_ncdf_inverse(nStabilizersForG, pG, k / M)
	varianceG = binomial_inverse_variance(nStabilizersForG, pG, expectedG, M)
	# print(expectedG, varianceG, pG, nStabilizersForG)

	# expectedGPrime = expectedG * (1 - delta)
	expectedGPrime = binomial_ncdf_inverse(nStabilizersForGPrime, pG, k / M)
	# varianceGPrime = varianceG * (1 - delta) ^ 0.5

	gPrimeForError = (3 * degBit * E + E - delta * E) / (3 * degBit - 1 + delta)
	
	# Note: this gives an upper bound for error
	# Just return the probability that Pr[G'
	if gPrimeForError == expectedGPrime:
		return 1#chebyshevUpperBound(gPrimeForError + 1, expectedGPrime, varianceGPrime)
	elif gPrimeForError > expectedGPrime:
		return 1#chebyshevUpperBound(gPrimeForError, expectedGPrime, varianceGPrime)
	else:
		# Just use varianceG for a better upper bound
		return chebyshevLowerBound(gPrimeForError, expectedGPrime, varianceG)

def pr_G_and_E_error(n_E, p_E, E, degBit, delta, k, M, pG):
	prE = binomial_pdf(n_E, p_E, E)
	return prE * pr_G_error_given_E(E, degBit, degStable, delta, k, M, pG)


def calc_pr_indep_G_E_lt_v_for_k(degBit, stableDeg, delta, v, k, pError, nE, pE, pG, M):
	maxE = nE
	prLtV = 0
	for e in range(0, maxE):
		prLtV = prLtV + pr_G_and_E_error(nE, pE, e, degBit, delta, k, M, pG)
	return prLtV

def calc_pr_indep_G_E_lt_v(bitDeg, stableDeg, delta, v, kRange, pError, M):
	pG = 0.5 - 0.5 * (1 - 2 * pError)^stableDeg
	pE = 0.5 + 0.5 * (1 - 2 * pError)^stableDeg - (1 - pError)^stableDeg
	nG = floor(bitDeg * stableDeg)
	nE = floor(bitDeg * stableDeg)

	error_prob = 1
	for k in range(*kRange):
		tmp = calc_pr_indep_G_E_lt_v_for_k(bitDeg, stableDeg, delta, v, k, pError, nE, pE, pG, M)
		print(k, tmp)
		error_prob = error_prob * tmp
	return error_prob

# TODO: make some plots?? (After LaTeX to ensure that I have some soundness to my arguments)
error_prob = calc_pr_indep_G_E_lt_v(degBit, degStable, delta, 1/6, (1, 100), 0.05, 10000)
error_prob

# 1.48343651109348e-15

In [ ]:
# 1.48343651109348e-15

In [35]:
### WRONG METHOD BELOW
degBit = 11
degStable = 11
delta = 1/6

def binomial_pdf(n, p, x):
    return binomial(n, x) * p ^ x * (1 - p) ^ (n-x)

# Pr[X >= x]
def binomial_ncdf(n, p, x):
    probSum = 0.0
    for i in range(x, n + 1):
        probSum = probSum + binomial_pdf(n, p, i)
    return probSum

# phi(targetP) = Pr[X >= x], return x
def binomial_ncdf_inverse(n, p, targetP):
    for i in range(1, n):
        ncdf_p = binomial_ncdf(n, p, i)
        if ncdf_p < targetP:
            return i - 1
    return n


# https://stats.stackexchange.com/questions/394960/variance-of-normal-order-statistics
# Variance for quantile
# TODO: verify
def binomial_inverse_variance(N, p, inverse, M):
	return p * (1-p) / ((M+2) * (binomial_pdf(N, p, inverse)^2))

## Lower bound via taking advantage of discrete RV
# where V < mu
def chebyshevLowerBound(V, mu, variance):
	return min(variance / (mu - V)^2, 1)

## Upper bound via taking advantage of discrete RV
# where V > mu and we are looking for Pr[mu < V]
def chebyshevUpperBound(V, mu, variance):
	return 1 - min(variance / (mu - V)^2, 0)

def pr_G_error_given_E(E, degBit, degStable, delta, k, M, pG):
	# a lower bound for the expectation of G
	nStabilizersForG = degBit * degStable - degStable * E
	if nStabilizersForG <= 0:
		return 1
	expectedG = binomial_ncdf_inverse(nStabilizersForG, pG, k / M)
	varianceG = binomial_inverse_variance(nStabilizersForG, pG, expectedG, M)
	# print(expectedG, varianceG, pG, nStabilizersForG)

	expectedGPrime = expectedG * (1 - delta)
	varianceGPrime = varianceG * (1 - delta) ^ 0.5

	gPrimeForError = (3 * degBit * E + E - delta * E) / (3 * degBit - 1 + delta)
	
	# Note: this gives an upper bound for error
	# Just return the probability that Pr[G'
	if gPrimeForError == expectedGPrime:
		return 1#chebyshevUpperBound(gPrimeForError + 1, expectedGPrime, varianceGPrime)
	elif gPrimeForError > expectedGPrime:
		return 1#chebyshevUpperBound(gPrimeForError, expectedGPrime, varianceGPrime)
	else:
		return chebyshevLowerBound(gPrimeForError, expectedGPrime, varianceGPrime)

def pr_G_and_E_error(n_E, p_E, E, degBit, delta, k, M, pG):
	prE = binomial_pdf(n_E, p_E, E)
	return prE * pr_G_error_given_E(E, degBit, degStable, delta, k, M, pG)


def calc_pr_indep_G_E_lt_v_for_k(degBit, stableDeg, delta, v, k, pError, nE, pE, pG, M):
	maxE = nE
	prLtV = 0
	for e in range(0, maxE):
		prLtV = prLtV + pr_G_and_E_error(nE, pE, e, degBit, delta, k, M, pG)
	return prLtV

def calc_pr_indep_G_E_lt_v(bitDeg, stableDeg, delta, v, kRange, pError, M):
	pG = 0.5 - 0.5 * (1 - 2 * pError)^stableDeg
	pE = 0.5 + 0.5 * (1 - 2 * pError)^stableDeg - (1 - pError)^stableDeg
	nG = floor(bitDeg * stableDeg)
	nE = floor(bitDeg * stableDeg)

	error_prob = 1
	for k in range(*kRange):
		tmp = calc_pr_indep_G_E_lt_v_for_k(bitDeg, stableDeg, delta, v, k, pError, nE, pE, pG, M)
		print(k, tmp)
		error_prob = error_prob * tmp
	return error_prob

# TODO: make some plots?? (After LaTeX to ensure that I have some soundness to my arguments)
error_prob = calc_pr_indep_G_E_lt_v(degBit, degStable, delta, 1/6, (1, 10), 0.01, 1000)
error_prob

1 0.352973359326478
2 0.0853387115853901
3 0.0488442052541024
4 0.0419293686978078
5 0.0219213789375707
6 0.0133462849078472
7 0.0133462849078472
8 0.0133391426016529
9 0.00642385499978010


2.06410449056573e-14